In [1]:
import numpy as np
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import os
sys.path.append('../quantnn-keras/layers/')

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.losses import squared_hinge
from keras.layers import Flatten, Dense, Activation, BatchNormalization, Conv1D

from keras.models import Sequential, Model
import numpy as np

from binary_layers import BinaryConv2D, BinaryConv1D
from binary_ops import binary_tanh as binary_tanh_op
from keras.regularizers import l2

Using TensorFlow backend.


In [ ]:
data_input_file = 'data/LOSO/USCHAD.npz'

tmp = np.load(data_input_file)
X = tmp['X']
y = tmp['y']
folds = tmp['folds']

n_class = y.shape[1]

avg_acc = []
avg_recall = []
avg_f1 = []
y = np.argmax(y, axis=1)

print('ConvNet Template 2017 {}'.format(data_input_file))

for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    X_train = X_train.squeeze()
    X_test = X_test.squeeze()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    # Create Model
    model = Sequential()

    _momentum = 0.1
    _epsilon = 0.001
    
    model.add(
        BinaryConv1D(
            kernel_size=30,
            filters=64,
            padding='same',
            input_shape=(250,23),
            kernel_lr_multiplier=10,use_bias=False))
    model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))
    model.add(Activation(binary_tanh_op))


    model.add(
        BinaryConv1D(
            kernel_size=30, filters=128, padding='same',
            kernel_lr_multiplier=10,use_bias=False))
    model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))
    model.add(Activation(binary_tanh_op))


    model.add(
        BinaryConv1D(
            kernel_size=30, filters=256, padding='same',
            kernel_lr_multiplier=10,use_bias=False))
    model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))
    model.add(Activation(binary_tanh_op))

    model.add(Flatten())
    model.add(Dense(12, use_bias=False))
    model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))

    # Train
    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss=squared_hinge, optimizer=adam, metrics=['accuracy'])
    #model.summary()

    model.fit(
        X_train, y_train, batch_size=50, epochs=100, validation_data=(X_test,y_test), verbose=0 )
    print (model.evaluate(X_test,y_test))

In [3]:
data_input_file = 'data/LOSO/USCHAD.npz'

tmp = np.load(data_input_file)
X = tmp['X']
y = tmp['y']
folds = tmp['folds']

n_class = y.shape[1]

avg_acc = []
avg_recall = []
avg_f1 = []
y = np.argmax(y, axis=1)

print('ConvNet Template 2017 {}'.format(data_input_file))

i = 0
train_idx = folds[i][0]
test_idx = folds[i][1]

X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test = X[test_idx], y[test_idx]
X_train = X_train.squeeze()
X_test = X_test.squeeze()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print (X_train.shape, X_test.shape)
print (y_train.shape, y_test.shape)

ConvNet Template 2017 data/LOSO/USCHAD.npz
(8931, 500, 6) (893, 500, 6)
(8931, 12) (893, 12)


In [4]:
print (folds.shape, len(folds))

(14, 2) 14


In [5]:
print('ConvNet Template 2017 {}'.format(data_input_file))

i = 0
train_idx = folds[i][0]
test_idx = folds[i][1]

X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test = X[test_idx], y[test_idx]
X_train = X_train.squeeze()
X_test = X_test.squeeze()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Create Model
model = Sequential()

model.add(
    BinaryConv1D(
        kernel_size=10,
        filters=64,
        padding='same',
        input_shape=(500,6),
        kernel_lr_multiplier=10,use_bias=False))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation(binary_tanh_op))


model.add(
    BinaryConv1D(
        kernel_size=10, filters=128, padding='same',
        kernel_lr_multiplier=10,use_bias=False))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation(binary_tanh_op))


model.add(
    BinaryConv1D(
        kernel_size=10, filters=128, padding='same',
        kernel_lr_multiplier=10,use_bias=False))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation(binary_tanh_op))

model.add(Flatten())
model.add(Dense(12, use_bias=False))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))

# Train
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss=squared_hinge, optimizer=adam, metrics=['accuracy'])
model.summary()

model.fit(
    X_train, y_train, batch_size=128, epochs=100, validation_data=(X_test,y_test), verbose=1 )
model.evaluate(X_test,y_test)

ConvNet Template 2017 data/LOSO/USCHAD.npz
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
binary_conv1d_1 (BinaryConv1 (None, 500, 64)           3840      
_________________________________________________________________
batch_normalization_1 (Batch (None, 500, 64)           256       
_________________________________________________________________
activation_1 (Activation)    (None, 500, 64)           0         
_________________________________________________________________
binary_conv1d_2 (BinaryConv1 (None, 500, 128)          81920     
_________________________________________________________________
batch_normalization_2 (Batch (None, 500, 128)          512       
_________________________________________________________________
activation_2 (Activation)    (None, 500, 128)          0         
___________________________

Epoch 42/100
8931/8931 [==============================] - 75s 8ms/step - loss: 0.9167 - acc: 0.7194 - val_loss: 0.9186 - val_acc: 0.4188
Epoch 43/100
8931/8931 [==============================] - 75s 8ms/step - loss: 0.9167 - acc: 0.7318 - val_loss: 0.9178 - val_acc: 0.5476
Epoch 44/100
8931/8931 [==============================] - 75s 8ms/step - loss: 0.9167 - acc: 0.7274 - val_loss: 0.9183 - val_acc: 0.5633
Epoch 45/100
8931/8931 [==============================] - 75s 8ms/step - loss: 0.9167 - acc: 0.7326 - val_loss: 0.9178 - val_acc: 0.5230
Epoch 46/100
8931/8931 [==============================] - 75s 8ms/step - loss: 0.9167 - acc: 0.7310 - val_loss: 0.9176 - val_acc: 0.5677
Epoch 47/100
8931/8931 [==============================] - 75s 8ms/step - loss: 0.9167 - acc: 0.7341 - val_loss: 0.9188 - val_acc: 0.4580
Epoch 48/100
8931/8931 [==============================] - 75s 8ms/step - loss: 0.9167 - acc: 0.7308 - val_loss: 0.9189 - val_acc: 0.4412
Epoch 49/100
8931/8931 [=================

[0.9176030285814976, 0.5722284435157947]

In [6]:
data_input_file = 'data/LOSO/USCHAD.npz'

tmp = np.load(data_input_file)
X = tmp['X']
y = tmp['y']
folds = tmp['folds']

n_class = y.shape[1]

avg_acc = []
avg_recall = []
avg_f1 = []
y = np.argmax(y, axis=1)

print('ConvNet Template 2017 {}'.format(data_input_file))

i = 0
train_idx = folds[i][0]
test_idx = folds[i][1]

X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test = X[test_idx], y[test_idx]
X_train = X_train.squeeze()
X_test = X_test.squeeze()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Create Model
model = Sequential()

model.add(
    Conv1D(
        kernel_size=10,
        filters=64,
        padding='same',
        input_shape=(500,6)))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation('relu'))
model.add(Conv1D(
        kernel_size=10, filters=64, padding='same'))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(12, use_bias=False))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))

# Train
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss=squared_hinge, optimizer=adam, metrics=['accuracy'])

model.summary()
model.fit(
    X_train, y_train, batch_size=50, epochs=100, validation_data=(X_test,y_test))
model.evaluate(X_test,y_test)

ConvNet Template 2017 data/LOSO/USCHAD.npz
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 500, 64)           3904      
_________________________________________________________________
batch_normalization_5 (Batch (None, 500, 64)           256       
_________________________________________________________________
activation_4 (Activation)    (None, 500, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 64)           41024     
_________________________________________________________________
batch_normalization_6 (Batch (None, 500, 64)           256       
_________________________________________________________________
activation_5 (Activation)    (None, 500, 64)           0         
_________________________________________________________________
flatten_2 (Flatten)          (Non

8931/8931 [==============================] - 18s 2ms/step - loss: 0.9167 - acc: 0.7246 - val_loss: 0.9167 - val_acc: 0.7077
Epoch 50/100
8931/8931 [==============================] - 18s 2ms/step - loss: 0.9167 - acc: 0.7345 - val_loss: 0.9168 - val_acc: 0.7380
Epoch 51/100
8931/8931 [==============================] - 18s 2ms/step - loss: 0.9167 - acc: 0.7340 - val_loss: 0.9172 - val_acc: 0.7010
Epoch 52/100
8931/8931 [==============================] - 18s 2ms/step - loss: 0.9167 - acc: 0.7151 - val_loss: 0.9173 - val_acc: 0.7077
Epoch 53/100
8931/8931 [==============================] - 18s 2ms/step - loss: 0.9167 - acc: 0.7375 - val_loss: 0.9167 - val_acc: 0.7346
Epoch 54/100
8931/8931 [==============================] - 18s 2ms/step - loss: 0.9167 - acc: 0.7372 - val_loss: 0.9168 - val_acc: 0.7548
Epoch 55/100
8931/8931 [==============================] - 18s 2ms/step - loss: 0.9167 - acc: 0.7508 - val_loss: 0.9172 - val_acc: 0.6114
Epoch 56/100
8931/8931 [==============================

[0.9166673230297221, 0.7939529676336591]